In [1]:
import requests
import os
import json
from base64 import b64encode

In [2]:
def get_new_token(response):
#     return response.headers.getlist('Set-Cookie')[0].decode("utf-8").split(";")[0].split('=')[1]
    return response.headers['Set-Cookie'].split(";")[0].split('=')[1]

In [3]:
def get_token_api():
    url = 'http://localhost:5000/token'
    r = requests.get(url)
    return r.text

In [4]:
cert_path = os.environ['GOSZAKUP_CERT']
cert_password = os.environ['GOSZAKUP_PW']

In [5]:
def sign_file(path):
    # print('sign file...')
    # document_id = response.meta['document_id']

    with open (cert_path, 'rb') as f:
        cert = b64encode(f.read()).decode('utf-8')

#     file_content = b64encode(response.body).decode('utf-8')

    with open (path, 'rb') as f:
        file_content = b64encode(f.read()).decode('utf-8')

    data = {
        'version': '2.0',
        'method': 'cms.sign',
        "params": {
            "data": file_content,
            "p12array": [
                {
                    "p12": cert,
                    "password": cert_password
                }
            ]
        }
    }


    headers = {
        'Content-Type': 'application/json'
    }

#     request = scrapy.http.JsonRequest(
#         method='POST',
#         url='http://127.0.0.1:14579/', 
#         data=data, 
#         headers=headers,
#         dont_filter=True,
#         # callback=cb
#     )

    r = requests.post(
        url='http://127.0.0.1:14579/', 
        data=json.dumps(data), 
        headers=headers,
    )
    
    return json.loads(r.text)['cms']

In [6]:
with open('/home/necuk/Desktop/zakup_kz/temp/uploading/filetxt.har') as f:
    tt = json.loads(f.read())['log']

In [7]:
tt.keys()

dict_keys(['version', 'creator', 'pages', 'entries', '_tlsErrors'])

In [8]:
d = tt['entries'][2]['request']
d['url']

'https://v3bl.goszakup.gov.kz/ru/files/upload_file_new'

In [42]:
# ######## FILE UPLOAD THAT WORKS.


# # headers = {h['name']: h['value'] for h in d['headers']}

# headers = {
#     'Content-Type': 'multipart/form-data; boundary=17ff5c6bf01',
#     'User-Agent': 'Java/1.8.0_222',
#     'Host': 'v3bl.goszakup.gov.kz',
#     'Accept': 'text/html, image/gif, image/jpeg, *; q=.2, */*; q=.2',
#     'Connection': 'keep-alive',
#     'Content-Length': '5765'
# }


# r_upload = requests.post(
#     url = d['url'],
#     headers = headers,
#     data = d['postData']['text'],
#     verify=False
# )

# r_upload

/home/necuk/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'v3bl.goszakup.gov.kz'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<Response [200]>

In [58]:
application_id = 37239610
app_token = '6e65abd9257fec6b3a0ab02a38e7d7af'

In [111]:
from requests_toolbelt.multipart.encoder import MultipartEncoder

# path = '../misc/file.txt'
path = 'mysheet.pdf'

file_name = os.path.basename(path)

multipart_encoder = MultipartEncoder(
    fields={
        "signature": cms,
        "token": app_token,
#         "Content-Type": "application/octet-stream",
        "userfile": (
#             file_name, open(path, 'rb'), 'application/octet-stream'
            file_name, open(path, 'rb'), 'application/octet-stream'
        )
    },
    boundary='17fb20d5188'
)

# post_data2 = multipart_encoder.to_string().decode('utf-8')
# post_data2 = post_data2.replace('filename=', 'fileName=').replace('name="file"', 'name="userfile"')

post_data2 = multipart_encoder.to_string()

In [112]:
print(post_data2)

b'--17fb20d5188\r\nContent-Disposition: form-data; name="signature"\r\n\r\nMIIGYAYJKoZIhvcNAQcCoIIGUTCCBk0CAQExDjAMBggqgw4DCgEDAQUAMBgGCSqGSIb3DQEHAaALBAl0ZXN0X2ZpbGWgggTaMIIE1jCCBICgAwIBAgIULBFucGgIEwDyHT8jjFMCo2xU7H4wDQYJKoMOAwoBAQECBQAwUzELMAkGA1UEBhMCS1oxRDBCBgNVBAMMO9Kw0JvQotCi0KvSmiDQmtCj05jQm9CQ0J3QlNCr0KDQo9Co0Ksg0J7QoNCi0JDQm9Cr0pogKEdPU1QpMB4XDTIxMDYwNDA3NTE1NVoXDTIyMDYwNDA3NTE1NVowggF1MSowKAYDVQQDDCHQmtCQ0JnQk9Ce0KDQntCU0KbQldCS0JAg0JzQkNCZ0K8xITAfBgNVBAQMGNCa0JDQmdCT0J7QoNCe0JTQptCV0JLQkDEYMBYGA1UEBRMPSUlOODMwNTA0NDAwMDQ5MQswCQYDVQQGEwJLWjGBnDCBmQYDVQQKDIGR0KLQntCS0JDQoNCY0KnQldCh0KLQktCeINChINCe0JPQoNCQ0J3QmNCn0JXQndCd0J7QmSDQntCi0JLQldCi0KHQotCS0JXQndCd0J7QodCi0KzQriAi0JjQndCW0JjQndCY0KDQmNCd0JPQntCS0JDQryDQmtCe0JzQn9CQ0J3QmNCvICLQm9CY0JTQldCgIjEYMBYGA1UECwwPQklOMTEwOTQwMDA0OTA5MSMwIQYDVQQqDBrQodCi0JDQndCY0KHQm9CQ0JLQntCS0J3QkDEfMB0GCSqGSIb3DQEJARYQQ0VPQExFQURFUkVORy5LWjBsMCUGCSqDDgMKAQEBATAYBgoqgw4DCgEBAQEBBgoqgw4DCgEDAQEAA0MABEDk36lRIduJHXYf2mZSoaY46q06pz8fTkInD91Y8hQSb

In [113]:
# path = '../misc/file.txt'
# path = '/home/necuk/Downloads/Resolution.pdf'
# path = '/home/necuk/Desktop/zakup_kz/temp/mysheet.pdf'

file_name = os.path.basename(path)

cms = sign_file(path)

application_id = 37239610

# with open(path, encoding='latin-1') as f:
#     file_content = f.read()

with open(path, 'rb') as f:
    file_content = f.read()

data_dummy = (
    '--17fb20d5188\r\nContent-Disposition: form-data; name="signature"\r\n\r\n{}\r\n--17fb20d5188\r\n'
    'Content-Disposition: form-data; name="token"\r\n\r\n{}\r\n--17fb20d5188\r\n'
    'Content-Disposition: form-data; name="userfile"; fileName="{}"\r\n'
    'Content-Type: application/octet-stream\r\n\r\n{}\r\n--17fb20d5188--\r\n'
)

post_data = data_dummy.format(
    cms, app_token, file_name, file_content
)


headers = {
    'Content-Type': 'multipart/form-data; boundary=17fb20d5188',
    'User-Agent': 'Java/1.8.0_222',
    'Host': 'v3bl.goszakup.gov.kz',
    'Accept': 'text/html, image/gif, image/jpeg, *; q=.2, */*; q=.2',
    'Connection': 'keep-alive',
    # 'Content-Length': str(len(post_data))
}


r_upload = requests.post(
    url = 'https://v3bl.goszakup.gov.kz/ru/files/upload_file_new',
    headers = headers,
#     data = post_data,
    data = post_data2,
    verify=False
)

if r_upload.status_code == 200:
    print(r_upload.text)
else:
    print(r_upload.status_code)

/home/necuk/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'v3bl.goszakup.gov.kz'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{"error":0,"file_id":"164939367","file_name":"mysheet.pdf","file_size":13191}


In [114]:
cookies = {
    'ci_session': get_new_token(r_upload),
}

headers = {
    'Connection': 'keep-alive',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'X-Requested-With': 'XMLHttpRequest',
    'sec-ch-ua-mobile': '?0',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36',
    'sec-ch-ua-platform': '"Linux"',
    'Origin': 'https://v3bl.goszakup.gov.kz',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Dest': 'empty',
    'Referer': f'https://v3bl.goszakup.gov.kz/ru/application/show_doc/7082536/{application_id}/19',
    'Accept-Language': 'en-US,en;q=0.9',
}

data = {
    'file_id_ar[0]': json.loads(r_upload.text)['file_id'],
}

r_check = requests.post(
    'https://v3bl.goszakup.gov.kz/files/check_status/', 
    headers=headers, 
    cookies=cookies, 
    data=data,
    verify=False
)

r_check.text

/home/necuk/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'v3bl.goszakup.gov.kz'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


'{"164939367":{"av":{"need":1,"check":1,"text":"Угроз не обнаружено"},"signature":{"need":1,"check":1,"text":"Проверки успешно пройдены"},"name":"mysheet.pdf","size":13191,"final":1}}'

In [115]:
cookies = {
    'show_filter_app': 'block',
#     'ci_session': token
#     'ci_session': get_new_token(r_upload)
    'ci_session': get_token_api()
}

headers = {
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
    'Upgrade-Insecure-Requests': '1',
    'Origin': 'https://v3bl.goszakup.gov.kz',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-User': '?1',
    'Sec-Fetch-Dest': 'document',
    'Referer': f'https://v3bl.goszakup.gov.kz/ru/application/show_doc/7196072/{application_id}/19',
    'Accept-Language': 'en-US,en;q=0.9',
}

data = {
    'send_form': '',
    'userfile[19][]': json.loads(r_upload.text)['file_id'],
#     'userfile[19][]': '163630171'
}

response = requests.post(
    f'https://v3bl.goszakup.gov.kz/ru/application/show_doc/7196072/{application_id}/19', 
    headers=headers, 
    cookies=cookies, 
    data=data,
    verify=False
)

print(response.status_code)

'Проверки успешно пройдены' in response.text

/home/necuk/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'v3bl.goszakup.gov.kz'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/necuk/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'v3bl.goszakup.gov.kz'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


200


True